## **Overview of Multiclass Classification: one-vs-all and all-pairs**

Give an overview of the algorithm and describe its advantages and disadvantages.

#### <ins>One-vs-All

This algorithm creates a classifier for each class (3 classifiers if there exists 3 classes). For each classifier, it is responsible for predicting whether an input belongs to its corresponding class or not.

Each classifier is trained on the entire dataset with modifications corresponding to each classifier.\
The modification changes the dataset such that when you're training a classifier for class $3$, labels for all the other classes are modified to $-1$ and labels for the classifier's class are modified to $1$. (More details will be provided in the Representation section)

#### <ins>All-Pairs

This algorithm creates a classifier for each pair of classes. For each classifier, it is responsible for predicting whether a given input belongs to one class or the other.

Each classifier is trained a portion of the dataset with modifications corresponding to each classifier.\
First, each classifier is assigned portion of the dataset that contains the classes the classifier is predicting for. Then, the assigned data's classes are changed so that one class is assigned the label of $1$ and the other is assigned the label of $-1$.

#### <ins>Advantages and Disadvantages of Multiclass Classification

Multiclass classification algorithm is an algorithm that classifies an input that can belong to one of the multiple classes (more than two classes).\
For this project, we will be implementing One-vs-All and All-Pairs algorithms for the multiclass classification of the UCI Iris dataset.

Compared to a multiclass classification algorithm that inherently encompasses multiclass classification (output of model predicts multiclass),
the main advantages of One-vs-All and All-Pairs stems from the use of binary classifiers.\
Because of the binary classifiers to represent multiclass classification, these two algorithms have implementation simplicity and easy interpretability of the predictions.

Unfortunately, the disadvantages also stem from the use of binary classifiers.
- The binary classifiers do not have any knowledge that it is used for multiclass classification and therefore, does not have inherent understanding of the multiclass classification problem.
- Due to training classifier for each class, each classifier is trained on a class imbalanced dataset and may result in overfitting.
- Training multiple classifiers can be computationally expensive.

#### <ins>Misc.

In this final project, we will be using the UCI Iris dataset we encountered in our previous homework:\
[`https://archive.ics.uci.edu/dataset/53/iris`](https://archive.ics.uci.edu/dataset/53/iris)

What we will be comparing to:
[scikit-learn multiclass classification](https://scikit-learn.org/1.5/modules/multiclass.html#multilabel-classification)


### Representation: Logistic Regression

#### Binary Logistic Regression
Given sample's feature values $x \in \mathbb{R}^{d}$ and a label $y  \in \{0, 1\}$, binary classification of input $x$ is predicted through combination of affine function and sigmoid function.
$$ y = \langle w, x\rangle $$ $$\sigma (y) = \frac{1}{1 + e^{-y}}$$
Therefore, our hypothesis function defined on weights $w$ is
$$h_{w}(x) = \frac{1}{1 + e^{-\langle w, x\rangle}}$$

#### Multiclass Logistic Regression
Now, using the binary logistic regression defined above, we will define one-vs-all and all-pairs multiclass logistic regression algorithms.

Pseudocode for one-vs-all (from textbook):

Given inputs:\
training set $S = (x_1, y_1), ..., (x_m, y_m)$\
binary classifier - logistic regression $L$

$\text{foreach } i \in Y:$\
$\text{ let } S_i = (x_1, (-1)^{\mathbb{1}_{[y_1 \neq i]}}), ..., (x_m, (-1)^{\mathbb{1}_{[y_m \neq i]}})$\
$\text{ let } h_i = L(S_i)$

Predicts:\
$ h(x) \in argmax_{i \in Y }\text{ }h_i(x)$


Pseudocode for all-pairs (from textbook):

Given inputs:\
training set $S = (x_1, y_1), ..., (x_m, y_m)$\
binary classifier - logistic regression $L$

$\text{foreach } i,j \in Y \text{ such that } i < j$\
$\text{ initialize empty } S_{i,j}$\
$\text{ for } t = 1, ..., m$\
$\text{ }\text{ If } y_t = i \text{, then add } (x_t, 1) \text{ to } S_{i,j}$
$\text{ }\text{ If } y_t = j \text{, then add } (x_t, -1) \text{ to } S_{i,j}$
$\text{ let } h_{i,j} = L(S_{i,j}$

Predicts:\
$ h(x) \in argmax_{i \in Y }\text{ } (\Sigma_{j \in Y} \text{ sign}(j-i) h_{i,j}(x))$



### Loss: Logistic Loss + Regularization

The loss function of a Logistic Regression classifier over $k$ classes is the **log-loss**, also called **cross-entropy loss**. Since we will only use binary classifier, e.g. Binary Logistic Regression, in this project, only **Binary Log Loss** will be introduced in this section.

The Binary Log Loss on a sample of m data points, also called the Binary Cross Entropy Loss, is:
$$L_S(h) = -\frac{1}{m} \sum_{i=1}^m (y_i \log h(x_i) + (1 - y_i)\log (1 - h(x_i)))$$

The corresponding gradient of the Binary Log loss with respect to the model's wights is:
$$\frac{\partial L_S(h)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^m (h(x_i) - y_i)x_{ij}$$

We also implement the L2 norm of wights to adpot Tikhonov regularization into our loss function. The L2 norm of wights is:
$$\lambda||w||_2^2 = \lambda\sum_{i=1}^{d}w_i^2$$ 
And the gradient of the L2 term with respect to the model's weights is:
$$\frac{\partial \lambda\sum_{i=1}^{d}w_i^2}{\partial w_j} = 2\lambda w_j$$

In conclusion, the total loss function would be:
$$L_S(h) = -\frac{1}{m} \sum_{i=1}^m (y_i \log h(x_i) + (1 - y_i)\log (1 - h(x_i)))+ \lambda\sum_{i=1}^{d}w_i^2$$

### Optimizer

**One-vs-All** and **All-Pairs** are both strategies used to solve muticlass classification problems by utilizing binary classifiers. In this case, Stochastic Gradient Descent (Mini Batch) is a suitable choice.   
In gradient descent, the general formula for weight update is:
$$w_j = w_j - \alpha \cdot \frac{\partial L}{\partial w_j}$$  

For each batch of size $m$, the gradient of the binary log loss with respect to the weight is:
$$\frac{\partial L}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} (h(x_i) - y_i) \cdot x_{ij}$$  

If incorporate regularization (mentioned in the previous section), the total gradient becomes:
$$\frac{\partial L}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} (h(x_i) - y_i) \cdot x_{ij} + 2 \lambda w_j$$
Thus, the final weight update equation is: 
$$w_j = w_j - \alpha \cdot \left( \frac{\partial L}{\partial w_j} + 2 \lambda w_j \right)$$  

Due to the nature of One-vs-All and All-pairs strategies, we apply this optimizer differently compared to direct multiclass classification techniques, such as multiclass logistic regression.  
**One-vs-All**: for each class $j$, you train a seperate binary classifier that distinguishes class $j$ from all other classes.  
**All-pairs**: for each unique class pair $(i, j)$, you train a binary classifier that differentiates between those two classes.  

#### Pseudocode: Stochastic Gradient Descent for Logistic Regression (Lecture 6 Slide 21)  
**Inputs**: Traning examples $S$, step size $\alpha$, batch size $b < |S|$  
Set converged false  
**while** not converged:  
&nbsp;&nbsp;&nbsp;&nbsp;Randomly shuffle $S$  
&nbsp;&nbsp;&nbsp;&nbsp;**for** $i = 0$ to $|S|/b - 1$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$S'$ = Extracted current batch using $i$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\mathbf{w} = \mathbf{w} - \alpha \cdot \nabla L_{S'}(h_w)$ + regularization  
&nbsp;&nbsp;&nbsp;&nbsp;converged = check_convergence$(S,w)$  
return

## **Model: one-vs-all**

In [2]:
import numpy as np

## **Model: all-pairs**

In [ ]:
import numpy as np

## **Check Model**

In [ ]:
import numpy as np

## **Main**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import numpy as np

# Compare performance with sklearn on iris dataset